In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
sns.set_theme()


number of unique patients
number of unique hospital stays
number of unique icu stays
gender distribution (hospital stay)
length of stay (hospital)
length of stay (icu)
Age at admission (hosp)
Age at admission (icu)
death distribution (hosp)
death distribution (icu)

# MIMIC

## general

In [ ]:
patients = pd.read_csv('data/raw/mimic/hosp/patients.csv.gz')
patients['anchor_year'] = patients['anchor_year'].astype(int)
print("number of unique patients:", len(patients['subject_id'].unique()))

## hospital

In [ ]:
admissions = pd.read_csv('data/raw/mimic/hosp/admissions.csv.gz')
admissions['admittime'] = pd.to_datetime(admissions['admittime'])
admissions['dischtime'] = pd.to_datetime(admissions['dischtime'])
print("number of unique hospital stays:",len(admissions['hadm_id'].unique()))
print("number of unique patients with hospital stay:",len(admissions['subject_id'].unique()))

admissions = pd.merge(patients,admissions,how='inner', on=['subject_id'])
print("gender distribution for hospital stays")
display(admissions.groupby(['gender']).count()[['hadm_id']])
print("death distribution for hospital stays")
display(admissions[['hospital_expire_flag','hadm_id']].drop_duplicates().groupby(['hospital_expire_flag']).count()[['hadm_id']])

In [ ]:
fs = 12 

admissions['temp'] = pd.to_datetime(admissions['anchor_year'].astype(str))
admissions['temp'] = admissions['admittime']  - admissions['temp'] 
admissions['temp'] = admissions['temp'].dt.days / 365.25
admissions['age_at_admission'] = admissions['anchor_age'] + admissions['temp']
median_age_at_admission = admissions['age_at_admission'].median()
sns.histplot(admissions['age_at_admission'], bins=100)
plt.axvline(median_age_at_admission, color='r', linestyle='--', label=f'Median: {median_age_at_admission:.1f} years')
plt.legend()
plt.title('Distribution of age at hospital admission')
plt.xlabel('Age in years')
plt.ylabel('Number of cases')
plt.show()


admissions['los'] = (admissions['dischtime'] - admissions['admittime']).dt.total_seconds() / 86400
median_los = admissions['los'].median()
admissions_rest = admissions[(admissions['los'] >= 0) & (admissions['los'] <= 100)]
sns.histplot(admissions_rest['los'], bins=100) #kde=True
plt.axvline(median_los, color='r', linestyle='--', label=f'Median: {median_los:.1f} days')
plt.legend()
plt.title('Distribution of length of hospital stay for first 100 days')
plt.xlabel('Days')
plt.ylabel('Number of cases')
plt.show()

#anchor age + (admittime - anchor year) = age at admission
icustays = pd.read_csv('data/raw/mimic/icu/icustays.csv.gz')
hosp_icu = pd.merge(admissions, icustays, how='inner', on='hadm_id')
median_age_at_admission = hosp_icu['age_at_admission'].median()
sns.histplot(hosp_icu['age_at_admission'], bins=100)
plt.axvline(median_age_at_admission, color='r', linestyle='--', label=f'Median: {median_age_at_admission:.1f} years')
plt.legend()
plt.title('Distribution of age at hospital admission with icu stay')
plt.xlabel('Age in years')
plt.ylabel('Number of cases')
plt.show()

hosp_icu['los'] = (hosp_icu['dischtime'] - hosp_icu['admittime']).dt.total_seconds() / 86400
median_los = hosp_icu['los'].median()
hosp_icu_rest = hosp_icu[(hosp_icu['los'] >= 0) & (hosp_icu['los'] <= 100)] #
sns.histplot(hosp_icu_rest['los'], bins=100) #kde=True
plt.axvline(median_los, color='r', linestyle='--', label=f'Median: {median_los:.1f} days')
plt.legend()
#plt.title('Distribution of length of hospital stay for first 100 days (with icu stay)')
plt.xlabel('Days', fontsize=fs)
plt.ylabel('Number of cases', fontsize=fs)
plt.legend(fontsize=fs)
plt.xticks(fontsize=fs)
plt.yticks(fontsize=fs)
plt.xlim(0, 100)
plt.tight_layout()
plt.savefig('images/data/mimic_hosp_los.png')
plt.show()

In [ ]:
micro = pd.read_csv("data/raw/mimic/hosp/microbiologyevents.csv.gz")
micro = micro[(micro['spec_type_desc'] == 'BLOOD CULTURE') & (micro['org_name'] != 'CANCELLED')]
micro['isPositive'] = micro['org_name'].notnull()

grouped = micro.groupby('hadm_id')['isPositive'].agg(['sum', 'size'])

at_least_one_positive = len(grouped[grouped['sum'] > 0])
all_negative = len(grouped[grouped['sum'] == 0])

print("total cases with blood culture",len(micro['hadm_id'].unique()))
print(f"Hospitalisationen mit mindestens einem positiven Test: {at_least_one_positive}")
print(f"Hospitalisationen mit allen negativen Tests: {all_negative}")

## icu

In [ ]:
icustays = pd.read_csv('data/raw/mimic/icu/icustays.csv.gz')
icustays['intime'] = pd.to_datetime(icustays['intime'])
icustays['outtime'] = pd.to_datetime(icustays['outtime'])

print("number of unique icu stays ",len(icustays['stay_id'].unique()))
print("number of unique patients with icu stay",len(icustays['subject_id'].unique()))
print("number of unique hospital stays with icu stay",len(icustays['hadm_id'].unique()))



In [ ]:
icustays = pd.merge(patients,icustays,how='inner', on=['subject_id'])
print("gender distribution for icu stays")
display(icustays.groupby(['gender']).count()[['stay_id']])
icustays['temp'] = pd.to_datetime(icustays['anchor_year'].astype(str))
icustays['temp'] = icustays['intime']  - icustays['temp'] 
icustays['temp'] = (icustays['temp'].dt.total_seconds() / 86400) / 365.25
icustays['age_at_admission'] = icustays['anchor_age'] + icustays['temp']
median_age_at_admission = icustays['age_at_admission'].median()
sns.histplot(icustays['age_at_admission'], bins=100)
plt.axvline(median_age_at_admission, color='r', linestyle='--', label=f'Median: {median_age_at_admission:.1f} years')
plt.legend()
plt.legend(fontsize=fs)
plt.xlabel('Age in years', fontsize=fs)
plt.ylabel('Number of cases', fontsize=fs)
plt.xticks(fontsize=fs)
plt.yticks(fontsize=fs)
plt.xlim(0, 100)
plt.tight_layout()
plt.savefig('images/data/mimic_age.png')
plt.show()

median_los = icustays['los'].median()
icustays_rest = icustays[(icustays['los'] >= 0)& (icustays['los'] <= 100)] # 
sns.histplot(icustays_rest['los'], bins=100) #kde=True
plt.axvline(median_los, color='r', linestyle='--', label=f'Median: {median_los:.1f} days')
plt.legend()
plt.legend(fontsize=fs)
plt.xlabel('Days', fontsize=fs)
plt.ylabel('Number of cases', fontsize=fs)
plt.xticks(fontsize=fs)
plt.yticks(fontsize=fs)
plt.xlim(0, 100)
plt.tight_layout()
plt.savefig('images/data/mimic_icu_los.png')
plt.show()


print("death distribution (with icu stay)")
df = pd.merge(icustays, admissions, how='inner', on='hadm_id')
display(df[['hospital_expire_flag','hadm_id']].drop_duplicates().groupby(['hospital_expire_flag']).count()[['hadm_id']])



In [ ]:
micro_icu = pd.merge(icustays, micro, how='inner', on='hadm_id')
micro_icu = micro_icu[(micro_icu['intime'] <= micro_icu['charttime']) & (micro_icu['charttime'] <= micro_icu['outtime'])]


grouped = micro_icu.groupby('stay_id')['isPositive'].agg(['sum', 'size'])

at_least_one_positive = len(grouped[grouped['sum'] > 0])
all_negative = len(grouped[grouped['sum'] == 0])

print("total icu stays without blood culture",len(icustays['stay_id'].unique())-len(micro_icu['stay_id'].unique()))
print("total icu stays with blood culture",len(micro_icu['stay_id'].unique()))
print(f"ICU stays mit mindestens einem positiven Test: {at_least_one_positive}")
print(f"ICU stays mit allen negativen Tests: {all_negative}")


In [ ]:

def ab_administration(database):

    events = pd.read_parquet('data/concepts/'+database+'/DrugAdministrationEvent.parquet').rename(columns={'SubjectPseudoIdentifier':'ID', 'StartDateTime':'Time'})

    stays = pd.read_parquet('data/concepts/'+database+'/AdministrativeCase.parquet').rename(columns={'SubjectPseudoIdentifier':'ID'})
    icu_stays = stays[stays['Location'] == 'icu'].copy()
    hosp_stays = stays[stays['Location'] == 'hosp'].copy()

    df = pd.merge(icu_stays,hosp_stays,how='inner',on='ID', suffixes=('_icu', '_hosp'))
    df = df[(df['AdmissionDateTime_hosp'] <= df['AdmissionDateTime_icu']) & (df['DischargeDateTime_icu'] <= df['DischargeDateTime_hosp'])]
    df.drop(['Location_icu','Location_hosp'],axis=1,inplace=True)
    df

    def check_time(row):
        if row['Time'] < row['AdmissionDateTime_icu']:
            return 'pre'
        elif row['Time'] > row['DischargeDateTime_icu']:
            return 'post'
        elif row['AdmissionDateTime_icu'] <= row['Time'] and row['Time'] <= row['DischargeDateTime_icu']:
            return 'in'
        else:
            raise Exception("not possible")

    df = pd.merge(df, events, how='inner',on='ID')
    # filter to get all applications within the hospital stay
    df = df[(df['AdmissionDateTime_hosp'] <= df['Time']) & (df['Time'] <= df['DischargeDateTime_hosp'])].copy()

    df['pre_in_post_icu'] = df.apply(check_time, axis=1)
    display(df.groupby(['pre_in_post_icu']).count())

ab_administration('mimic')

# eicu

## icu

In [ ]:
patient = pd.read_csv('data/raw/eicu/patient.csv.gz')
patient['age'] = patient['age'].replace('> 89', '90')


print("number of unique icu stays: ",len(patient['patientunitstayid'].unique()))
print("number of unique patients with icu stay",len(patient['uniquepid'].unique()))
print("number of unique patients with hospital stay",len(patient['uniquepid'].unique()))
print("number of unique patients", len(patient['uniquepid'].unique()))


print('number of unique hospital stays with icu stay: ',len(patient['patienthealthsystemstayid'].unique()))
print('number of unique hospital stays: ',len(patient['patienthealthsystemstayid'].unique()))

print('gender distribution icu stays')
print(patient[['patientunitstayid','gender']].drop_duplicates().groupby(['gender']).count()[['patientunitstayid']])

In [ ]:
temp = patient[['patientunitstayid','age']].drop_duplicates()
temp = temp[temp['age'].notna()]
temp['age'] = temp['age'].astype(int)
median_age_at_admission = temp['age'].median()
sns.histplot(temp['age'], bins=90)
plt.axvline(median_age_at_admission, color='r', linestyle='--', label=f'Median: {median_age_at_admission:.1f} years')
plt.legend()
#plt.title('Distribution of age at ICU admission')
plt.xlabel('Age in years', fontsize=fs)
plt.ylabel('Number of cases', fontsize=fs)
plt.legend(fontsize=fs)
plt.xticks(fontsize=fs)
plt.yticks(fontsize=fs)
plt.xlim(0, 100)
plt.tight_layout()
plt.savefig('images/data/eicu_age.png')
plt.show()

patient = patient[patient['unitdischargeoffset'] >= 0]
patient = patient[patient['hospitaladmitoffset'] <= 0]
patient = patient[patient['hospitaldischargeoffset'] >= 0]
patient['hosp_los'] = pd.to_timedelta(patient['hospitaladmitoffset'].abs()+patient['hospitaldischargeoffset'].abs(), unit='minutes').dt.total_seconds() / 86400
patient['icu_los'] = pd.to_timedelta(patient['unitdischargeoffset'].abs(), unit='minutes').dt.total_seconds() / 86400
patient = patient[patient['hosp_los'] >= patient['icu_los']]

temp_rest = patient[['patienthealthsystemstayid','hosp_los']].drop_duplicates()
median_los = temp_rest['hosp_los'].median()
temp_rest = temp_rest[(temp_rest['hosp_los'] >= 0) & (temp_rest['hosp_los'] <= 100)] #
sns.histplot(temp_rest['hosp_los'], bins=100) #kde=True
plt.axvline(median_los, color='r', linestyle='--', label=f'Median: {median_los:.1f} days')
plt.legend()
#plt.title('Distribution of length of hospital stay for patients with icu stay')
plt.xlabel('Days', fontsize=fs)
plt.ylabel('Number of cases', fontsize=fs)
plt.legend(fontsize=fs)
plt.xticks(fontsize=fs)
plt.yticks(fontsize=fs)
plt.xlim(0, 100)
plt.tight_layout()
plt.savefig('images/data/eicu_hosp_los.png')
plt.show()

temp_rest = patient[['patientunitstayid','icu_los']].drop_duplicates()
median_los = temp_rest['icu_los'].median()
temp_rest = temp_rest[(temp_rest['icu_los'] >= 0) & (temp_rest['icu_los'] <= 100)] #
sns.histplot(temp_rest['icu_los'], bins=100) #kde=True
plt.axvline(median_los, color='r', linestyle='--', label=f'Median: {median_los:.1f} days')
plt.legend()
#plt.title('Distribution of length of ICU stay')
plt.xlabel('Days', fontsize=fs)
plt.ylabel('Number of cases', fontsize=fs)
plt.legend(fontsize=fs)
plt.xticks(fontsize=fs)
plt.yticks(fontsize=fs)
plt.xlim(0, 100)
plt.tight_layout()
plt.savefig('images/data/eicu_icu_los.png')
plt.show()


print("death distribution (with icu stay)")
display(patient[['patienthealthsystemstayid','hospitaldischargestatus']].drop_duplicates().groupby(['hospitaldischargestatus']).count()[['patienthealthsystemstayid']])


In [ ]:
df = pd.read_csv('data/raw/eicu/microLab.csv.gz')

patient = pd.read_csv('data/raw/eicu/patient.csv.gz')[['patientunitstayid','patienthealthsystemstayid']]
df = pd.merge(df,patient, how='inner', on='patientunitstayid')


micro = df[df['culturesite'].isin(['Blood, Venipuncture', 'Blood, Central Line'])].copy()
micro['isPositive'] = micro['organism'] != 'no growth'



grouped = micro.groupby('patientunitstayid')['isPositive'].agg(['sum', 'size'])


at_least_one_positive = len(grouped[grouped['sum'] > 0])


all_negative = len(grouped[grouped['sum'] == 0])

print("total icu stays without blood culture",len(patient['patientunitstayid'].unique())-len(micro['patientunitstayid'].unique()))
print("total icu stays with blood culture",len(micro['patientunitstayid'].unique()))
print(f"ICU stays mit mindestens einem positiven Test: {at_least_one_positive}")
print(f"ICU stays mit allen negativen Tests: {all_negative}")



In [ ]:
ab_administration('eicu')

# pic

## general

In [ ]:
patients = pd.read_csv('data/raw/pic/PATIENTS.csv.gz')
patients['DOB'] = pd.to_datetime(patients['DOB'])
#display(patients)
#patients['anchor_year'] = patients['anchor_year'].astype(int)
print("number of unique patients:", len(patients['SUBJECT_ID'].unique()))

# hospital

In [ ]:
admissions = pd.read_csv('data/raw/pic/ADMISSIONS.csv.gz')
admissions = pd.merge(patients,admissions,how='inner', on=['SUBJECT_ID'])
admissions['ADMITTIME'] = pd.to_datetime(admissions['ADMITTIME'])
admissions['DISCHTIME'] = pd.to_datetime(admissions['DISCHTIME'])

print("number of unique hospital stays:",len(admissions['HADM_ID'].unique()))
print("number of unique patients with hospital stay:",len(admissions['SUBJECT_ID'].unique()))

print("gender distribution for hospital stays")
display(admissions.groupby(['GENDER']).count()[['HADM_ID']])

print("death distribution")
display(admissions[['HOSPITAL_EXPIRE_FLAG','HADM_ID']].drop_duplicates().groupby(['HOSPITAL_EXPIRE_FLAG']).count()[['HADM_ID']])



In [ ]:
admissions['age_at_admission'] = ((admissions['ADMITTIME'] - admissions['DOB']).dt.total_seconds() / 86400) / 365.25
median_age_at_admission = admissions['age_at_admission'].median()
sns.histplot(admissions['age_at_admission'], bins=100)
plt.axvline(median_age_at_admission, color='r', linestyle='--', label=f'Median: {median_age_at_admission:.1f} years')
plt.legend()
plt.title('Distribution of age at hospital admission')
plt.xlabel('Age in years')
plt.ylabel('Number of cases')
plt.show()


admissions['los'] = (admissions['DISCHTIME'] - admissions['ADMITTIME']).dt.total_seconds() / 86400
median_los = admissions['los'].median()
admissions_rest = admissions[(admissions['los'] >= 0) & (admissions['los'] <= 100)] #
sns.histplot(admissions_rest['los'], bins=100) #kde=True
plt.axvline(median_los, color='r', linestyle='--', label=f'Median: {median_los:.1f} days')
plt.legend()
plt.title('Distribution of length of hospital stay')
plt.xlabel('Days')
plt.ylabel('Number of cases')
plt.show()


In [ ]:
micro = pd.read_csv("data/raw/pic/MICROBIOLOGYEVENTS.csv.gz")


micro = micro[micro['SPEC_ITEMID'].str.contains('LIS0162')]
#| micro['SPEC_ITEMID'].str.contains('LIS05088') | micro['SPEC_ITEMID'].str.contains('LIS0156')| micro['SPEC_ITEMID'].str.contains('LIS0567')


micro['isPositive'] = micro['ORG_ITEMID'].notnull()
#display(micro[micro['ORG_ITEMID'].isna()])
#display(micro)

grouped = micro.groupby('HADM_ID')['isPositive'].agg(['sum', 'size'])

at_least_one_positive = len(grouped[grouped['sum'] > 0])
all_negative = len(grouped[grouped['sum'] == 0])

print("total cases with blood culture",len(micro['HADM_ID'].unique()))
print(f"Hospitalisationen mit mindestens einem positiven Test: {at_least_one_positive}")
print(f"Hospitalisationen mit allen negativen Tests: {all_negative}")

## icu

In [ ]:
icustays = pd.read_csv('data/raw/pic/ICUSTAYS.csv.gz')
print("number of unique icu stays ",len(icustays['ICUSTAY_ID'].unique()))
print("number of unique patients with icu stay",len(icustays['SUBJECT_ID'].unique()))
print("number of unique hospital stays with icu stay",len(icustays['HADM_ID'].unique()))

In [ ]:
icustays = pd.read_csv('data/raw/pic/ICUSTAYS.csv.gz')
icustays['INTIME'] = pd.to_datetime(icustays['INTIME'])
icustays['OUTTIME'] = pd.to_datetime(icustays['OUTTIME'])
icustays = pd.merge(patients,icustays,how='inner', on=['SUBJECT_ID'])
print("gender distribution for icu stays")
display(icustays.groupby(['GENDER']).count()[['ICUSTAY_ID']])
icustays['age_at_admission'] = (icustays['INTIME'] - icustays['DOB']).dt.days / 365.25
median_age_at_admission = icustays['age_at_admission'].median()
sns.histplot(icustays['age_at_admission'], bins=100)
plt.axvline(median_age_at_admission, color='r', linestyle='--', label=f'Median: {median_age_at_admission:.1f} years')
plt.legend()
#plt.title('Distribution of age at ICU admission')
plt.xlabel('Age in years', fontsize=fs)
plt.ylabel('Number of cases', fontsize=fs)
plt.legend(fontsize=fs)
plt.xticks(fontsize=fs)
plt.yticks(fontsize=fs)
#plt.xlim(0, 100)
plt.tight_layout()
plt.savefig('images/data/pic_age.png')
plt.show()

median_los = icustays['LOS'].median()
icustays_rest = icustays[(icustays['LOS'] >= 0) & (icustays['LOS'] <= 100)] #
sns.histplot(icustays_rest['LOS'], bins=100) #kde=True
plt.axvline(median_los, color='r', linestyle='--', label=f'Median: {median_los:.1f} days')
plt.legend()
#plt.title('Distribution of length of icu stay')
plt.xlabel('Days', fontsize=fs)
plt.ylabel('Number of cases', fontsize=fs)
plt.legend(fontsize=fs)
plt.xticks(fontsize=fs)
plt.yticks(fontsize=fs)
plt.xlim(0, 100)
plt.tight_layout()
plt.savefig('images/data/pic_icu_los.png')
plt.show()

print("death distribution (with icu stay)")
df = pd.merge(icustays, admissions, how='inner', on='HADM_ID')
display(df[['HOSPITAL_EXPIRE_FLAG','HADM_ID']].drop_duplicates().groupby(['HOSPITAL_EXPIRE_FLAG']).count()[['HADM_ID']])

df['los'] = (df['DISCHTIME'] - df['ADMITTIME']).dt.total_seconds() / 86400
median_los = df['los'].median()
df_rest = df[(df['los'] >= 0) & (df['los'] <= 100)] #
sns.histplot(df_rest['los'], bins=100) #kde=True
plt.axvline(median_los, color='r', linestyle='--', label=f'Median: {median_los:.1f} days')
plt.legend()
#plt.title('Distribution of length of hospital stay (with icu stay)')
plt.xlabel('Days', fontsize=fs)
plt.ylabel('Number of cases', fontsize=fs)
plt.legend(fontsize=fs)
plt.xticks(fontsize=fs)
plt.yticks(fontsize=fs)
plt.xlim(0, 100)
plt.tight_layout()
plt.savefig('images/data/pic_hosp_los.png')
plt.show()

In [ ]:

micro_icu = pd.merge(icustays, micro, how='inner', on='HADM_ID')
micro_icu = micro_icu[(micro_icu['INTIME'] <= micro_icu['CHARTTIME']) & (micro_icu['CHARTTIME'] <= micro_icu['OUTTIME'])]


grouped = micro_icu.groupby('ICUSTAY_ID')['isPositive'].agg(['sum', 'size'])

at_least_one_positive = len(grouped[grouped['sum'] > 0])
all_negative = len(grouped[grouped['sum'] == 0])

print("total icu stays without blood culture",len(icustays['ICUSTAY_ID'].unique())-len(micro_icu['ICUSTAY_ID'].unique()))
print("total icu stays with blood culture",len(micro_icu['ICUSTAY_ID'].unique()))
print(f"ICU stays mit mindestens einem positiven Test: {at_least_one_positive}")
print(f"ICU stays mit allen negativen Tests: {all_negative}")


In [ ]:

ab_administration('pic')